Итак, чтобы построить рекомендательную систему на основе контента, необходимо:

Для каждого продукта создать характеризующие его признаки.
Найти показатель близости между всеми продуктами.
Порекомендовать пользователю продукты, которые показывают наибольшую близость с теми продуктами, которые он высоко оценил.
Давайте реализуем подобную рекомендательную систему на практике. Будем работать с датасетом, содержащим информацию об оценивании фильмов на платформе Netflix.

show_id — id фильма,
type — его тип (фильм или сериал),
title — название,
director — режиссер,
cast — актерский состав,
country — страна,
date_added — дата добавления,
release_year — год выхода на экраны,
rating — рейтинг,
duration — продолжительность,
listened_in — жанр(-ы),
description — описание.
В первую очередь нам необходимо определить, на основании чего мы будем рассматривать близость фильмов. Выберем для этой задачи описание фильма, ведь в нём, скорее всего, содержится много информации. Однако описание — это текст. Есть много подходов к преобразованию текста в вектор, и мы будем использовать подход TF-IDF (Term Frequency-Inverse Document Frequency).

Показатель TD-IDF — это индикатор того, насколько релевантно слово в контексте документа.

Этот показатель возрастает пропорционально количеству раз, когда слово встречается в тексте, и уменьшается пропорционально количеству слов во всех текстах в целом.

Таким образом:

Коэффициент будет выше, если слово характерно именно для этого текста, то есть встречается в данном тексте часто, но не встречается в других текстах.
Коэффициент будет ниже, если слово не встречается почти нигде или встречается одинаковое количество раз во всех текстах, то есть не характеризует никакой текст в отдельности.
Если вам интересно подробнее изучить алгоритм создания такого представления, рекомендуем прочитать статью.

Чтобы преобразовать текст по этому принципу, нам понадобится соответствующая функция из библиотеки sklearn — импортируем её:

Чтобы преобразовать текст по этому принципу, нам понадобится соответствующая функция из библиотеки sklearn — импортируем её:

In [2]:
from sklearn.feature_extraction.text import TfidfVectorizer

Далее учтём стоп-слова, т. е. предлоги и другие служебные части речи, которые не несут содержательной информации, и с учётом этого определим нашу модель:



In [3]:
model = TfidfVectorizer(stop_words='english')

Заполним пропуски пустыми строками:

In [6]:
import pandas as pd
df = pd.read_csv('/Users/egor/Documents/data_science_course/SKILLFACTORY/MATH&ML-15. Рекомендательные системы. Часть II/data/netflix_titles.csv')
df['description'] = df['description'].fillna('')


Трансформируем наши описания в матрицу:

In [7]:
feature_matrix = model.fit_transform(df['description'])

Задание 2.2

Сколько столбцов в получившейся матрице?

In [8]:
feature_matrix

<7787x17905 sparse matrix of type '<class 'numpy.float64'>'
	with 107187 stored elements in Compressed Sparse Row format>

Теперь необходимо вычислить косинусную близость. Можно сделать это так:

In [9]:
from sklearn.metrics.pairwise import linear_kernel
cosine_sim = linear_kernel(feature_matrix, feature_matrix)

Обратите внимание! Мы используем здесь linear_kernel(), а не cosine_similarity(), так как в косинусном расстоянии в знаменателе реализуется нормировка векторов, а TF-IDF создаёт уже нормализованные векторы.

Вернём индексацию и уберём дубликаты из данных:

In [11]:
indices = pd.Series(df.index,index=df['title']).drop_duplicates()

In [12]:
indices

title
3%                                            0
7:19                                          1
23:59                                         2
9                                             3
21                                            4
                                           ... 
Zozo                                       7782
Zubaan                                     7783
Zulu Man in Japan                          7784
Zumbo's Just Desserts                      7785
ZZ TOP: THAT LITTLE OL' BAND FROM TEXAS    7786
Length: 7787, dtype: int64

Теперь пропишем функцию для создания рекомендаций:

In [13]:
def get_recommendations(title):
    idx = indices[title]
    #вычисляем попарные коэффициенты косинусной близости
    scores = list(enumerate(cosine_sim[idx]))
    #сортируем фильмы на основании коэффициентов косинусной близости по убыванию
    scores = sorted(scores, key=lambda x: x[1], reverse=True)
    #выбираем десять наибольших значений косинусной близости; нулевую не берём, т. к. это тот же фильм
    scores =   scores[1:11]
    #забираем индексы
    ind_movie = [i[0] for i in scores]
    #возвращаем названия по индексам
    return df['title'].iloc[ind_movie]

Например, если мы хотим найти рекомендации по фильму "Star Trek", то функция будет выдавать следующий результат:

In [21]:
get_recommendations('Star Trek')

5788             Star Trek: The Next Generation
5787                      Star Trek: Enterprise
5786                 Star Trek: Deep Space Nine
5557                     She's Out of My League
134                                  7 Days Out
6664                        The Midnight Gospel
6023                                     Teresa
4863    Pinkfong & Baby Shark's Space Adventure
5104                                       Rats
5970                             Tales by Light
Name: title, dtype: object

Задание 2.3

Найдите вторую рекомендацию для детского фильма "Balto", вышедшего на экраны в 1995 году:

In [24]:
get_recommendations('Balto')

709                Balto 2: Wolf Quest
7446                           Vroomiz
1338    Chilling Adventures of Sabrina
7388                          Vampires
1770                          Dinotrux
2767                     Hold the Dark
5540                 Shanghai Fortress
4041                             Mercy
2582                       Half & Half
1365        Christmas in the Heartland
Name: title, dtype: object

ПРАКТИКА
Мы рассмотрели несколько вариантов коллаборативной фильтрации на простейших примерах, и теперь пришло время практики с настоящими данными. Сначала мы будем использовать подход memory-based в модификации item-based, а затем SVD. В результате применения обоих алгоритмов мы сможем сравнить получившееся качество.

Для создания алгоритмов рекомендательной системы будем использовать библиотеку surprise.

Установим её:

In [25]:
!pip install scikit-surprise

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 772.0/772.0 kB 11.7 MB/s eta 0:00:00a 0:00:01
  Preparing metadata (setup.py) ... done
  Created wheel for scikit-surprise: filename=scikit_surprise-1.1.3-cp311-cp311-macosx_11_0_arm64.whl size=1272063 sha256=fc52e133470176a66855ddd4b7bb8871b74d465e4b38c151af621ec5b807ec37
  Stored in directory: /Users/egor/Library/Caches/pip/wheels/f4/2b/26/e2a5eae55d3b7688995e66abe7f40473aac6c95ddd8ee174a8
Successfully built scikit-surprise


Примечание. Заметьте, что в названии библиотеки содержится название уже известного нам модуля scikit — это не просто так: названия функций и аргументов, да и сам принцип работы этих модулей похожи.

В нашей задаче мы будем использовать датасет movielens, который содержит информацию о фильмах и выставленных рейтингах с сайта https://movielens.org/.

Импортируем необходимые нам компоненты и считаем данные с помощью специального метода Reader:

In [27]:
from surprise import Dataset
from surprise import Reader
from surprise.dataset import BUILTIN_DATASETS #с помощью данного объекта мы можем использовать встроенные датасеты

data = Dataset.load_from_file(
    "/Users/egor/Documents/data_science_course/SKILLFACTORY/MATH&ML-15. Рекомендательные системы. Часть II/data/u.data.txt",
    reader=Reader(line_format="user item rating timestamp", sep="\t"),
)

Чтобы обучать рекомендательные системы с помощью surprise, мы создали объект Dataset. Объект surprise.dataset — это набор данных, который содержит следующие поля в указанном порядке:

- идентификаторы пользователей,
- идентификаторы элементов,
- соответствующая оценка.
Преобразуем данные к формату pandas DataFrame для удобной работы с ними:

In [30]:
df = pd.DataFrame(data.raw_ratings, columns=['userId', 'movieId', 'rating', 'timestamp'])
df

,userId,movieId,rating,timestamp
0,196,242,3.0,881250949
1,186,302,3.0,891717742
2,22,377,1.0,878887116
3,244,51,2.0,880606923
4,166,346,1.0,886397596
...,...,...,...,...
99995,880,476,3.0,880175444
99996,716,204,5.0,879795543
99997,276,1090,1.0,874795795
99998,13,225,2.0,882399156


В данных присутствуют следующие признаки:

userId — идентификаторы пользователей сайта movielens;
movieId — идентификаторы фильмов;
rating — оценки фильмов, выставленные пользователями по шкале от 1 до 5;
timestamp — время оценки фильма пользователем. Данный формат представления времени показывает, сколько секунд прошло с 1 января 1970 года.

Задание 3.1

Сколько уникальных фильмов в наборе данных?

In [31]:
df.movieId.nunique()

1682

Задание 3.2

Сколько уникальных пользователей в наборе данных?

In [32]:
df.userId.nunique()

943

Задание 3.3

Какая оценка встречается в наборе данных чаще всего? Введите ответ в виде целого числа.

In [33]:
df.rating.mode()

0    4.0
Name: rating, dtype: float64

Библиотека surprise очень похожа на библиотеку sklearn, и тоже позволяет разбить данные на обучающую и тестовую выборки всего одной функцией — surprise.model_selection.train_test_split().

Задание 3.4

Разбейте данные на обучающую и тестовую выборки. Объём тестовой выборки должен составлять 25 % от общего объёма данных. В качестве значения параметра random_state возьмите число 13.

Сколько объектов попало в тестовую выборку?

In [48]:
from surprise.model_selection import train_test_split

train_df, test_df = train_test_split(data, test_size=0.25, random_state=13)
len(test_df)

25000

Импортируем функции для построения рекомендательных систем (SVD — для model-based-подхода и KNNBasic — для memory-basic-подхода) и для оценки качества результата.

In [43]:
from surprise import SVD, KNNBasic, accuracy

Теперь реализуем обычную коллаборативную фильтрацию. Выберем оценку схожести через косинусную близость и item-based-подход:

In [45]:
sim_options = {
    'name': 'cosine',
    'user_based': False
}
 
knn = KNNBasic(sim_options=sim_options)

Обучим алгоритм:

In [49]:
knn.fit(train_df)

Computing the cosine similarity matrix...
Done computing similarity matrix.


Теперь давайте посмотрим, какие рекомендации мы получили, с помощью следующей программы:

In [50]:
predictions = knn.test(test_df)
predictions

[Prediction(uid='7', iid='633', r_ui=5.0, est=4.199452349030111, details={'actual_k': 40, 'was_impossible': False}),
 Prediction(uid='422', iid='287', r_ui=3.0, est=3.4703437660463736, details={'actual_k': 40, 'was_impossible': False}),
 Prediction(uid='804', iid='163', r_ui=3.0, est=3.5716736533692854, details={'actual_k': 40, 'was_impossible': False}),
 Prediction(uid='189', iid='480', r_ui=5.0, est=4.222825780855538, details={'actual_k': 40, 'was_impossible': False}),
 Prediction(uid='238', iid='546', r_ui=3.0, est=3.473417286928204, details={'actual_k': 17, 'was_impossible': False}),
 Prediction(uid='804', iid='216', r_ui=4.0, est=3.922551907749182, details={'actual_k': 40, 'was_impossible': False}),
 Prediction(uid='350', iid='204', r_ui=4.0, est=4.345238219480267, details={'actual_k': 38, 'was_impossible': False}),
 Prediction(uid='708', iid='993', r_ui=4.0, est=3.4458505791534115, details={'actual_k': 40, 'was_impossible': False}),
 Prediction(uid='193', iid='1078', r_ui=4.0, es

Информация о каждой паре будет содержать следующие характеристики:

uid — id пользователя;
iid — id элемента;
r_ui (float) — реальный рейтинг, который этот пользователь поставил этому элементу;
est (float) — предсказанный рейтинг.

Задание 3.5

1. Каков реальный рейтинг, выставленный пользователем с ID 500 для фильма с ID 699?


In [72]:
predictions_df = pd.DataFrame(predictions)
predictions_df[(predictions_df.uid == '500')&(predictions_df.iid == '699')]

,uid,iid,r_ui,est,details
946,500,699,3.0,3.47479,"{'actual_k': 40, 'was_impossible': False}"


Теперь необходимо вычислить RMSE для получившихся предсказаний:



In [73]:
accuracy.rmse(predictions)


RMSE: 1.0272


1.0271678039029761

Если округлить результат до сотых, получаем .

Итак, мы построили систему рекомендаций и даже оценили её качество. Но как же вывести рекомендации для конкретного пользователя?
Для начала давайте оформим наши предсказания в таблицу и отсортируем их по прогнозируемому рейтингу:

In [75]:
pred = pd.DataFrame(predictions)
pred.sort_values(by=['est'],inplace=True,ascending = False)
pred

,uid,iid,r_ui,est,details
22469,849,234,5.0,4.951929,"{'actual_k': 19, 'was_impossible': False}"
1974,849,427,4.0,4.950547,"{'actual_k': 19, 'was_impossible': False}"
8272,849,568,4.0,4.949215,"{'actual_k': 19, 'was_impossible': False}"
5138,849,174,5.0,4.947691,"{'actual_k': 19, 'was_impossible': False}"
22021,688,1127,5.0,4.928412,"{'actual_k': 15, 'was_impossible': False}"
...,...,...,...,...,...
15746,405,194,1.0,1.000000,"{'actual_k': 40, 'was_impossible': False}"
21245,405,197,4.0,1.000000,"{'actual_k': 40, 'was_impossible': False}"
13891,405,511,2.0,1.000000,"{'actual_k': 40, 'was_impossible': False}"
21639,181,151,2.0,1.000000,"{'actual_k': 40, 'was_impossible': False}"


Теперь мы можем вывести рекомендуемые для конкретного пользователя фильмы, начиная от наиболее релевантного (с точки зрения рекомендаций) и заканчивая наименее релевантным.



In [79]:
recom = pred[pred.uid =='849']['iid'].to_list()
recom

['234', '427', '568', '174']

Задание 3.6

Реализуйте user-based-алгоритм. Какое значение RMSE получилось для коллаборативной фильтрации типа user-based? Ответ округлите до двух знаков после точки-разделителя.

In [80]:
sim_options = {
    'name': 'cosine',
    'user_based': True
}
 
knn2 = KNNBasic(sim_options=sim_options)
knn2.fit(train_df)
predictions2 = knn2.test(test_df)
accuracy.rmse(predictions2)

Computing the cosine similarity matrix...
Done computing similarity matrix.
RMSE: 1.0175


1.0174852296380237

Задание 3.7

Теперь давайте сравним полученные результаты с результатами SVD-алгоритма. Реализуйте SVD с параметрами по умолчанию.

Какое значение RMSE получилось для SVD? Ответ округлите до двух знаков после точки-разделителя.

In [82]:
svd = SVD()
svd.fit(train_df)
predictions3 = svd.test(test_df)
accuracy.rmse(predictions3)

RMSE: 0.9416


0.9415733431844185

Гибридная РС — это особый тип рекомендательной системы, который представляет собой комбинацию из нескольких методов. Обычно это комбинация контентного подхода и коллаборативной фильтрации. Такое сочетание может помочь преодолеть недостатки, с которыми мы сталкиваемся при использовании этих методов по отдельности, а также в некоторых случаях может быть более эффективным.

Гибридные подходы к рекомендательным системам можно реализовать по-разному, например используя комбинацию из рекомендаций, полученных обеими системами, или используя взвешенную сумму предсказанных рейтингов.

Давайте на практике рассмотрим, как создать рекомендательную систему с помощью гибридного подхода.

Разумеется, можно комбинировать различные подходы самостоятельно, однако для удобства уже реализован модуль LightFM — установим библиотеку через следующую команду:

In [83]:
!pip install lightfm

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 316.4/316.4 kB 7.2 MB/s eta 0:00:00a 0:00:01
  Preparing metadata (setup.py) ... done
  Created wheel for lightfm: filename=lightfm-1.17-cp311-cp311-macosx_11_0_arm64.whl size=419472 sha256=8e4b3dc3927db757de7134e8a67f78efaf86ed9d9bd54ba8d37058b07fdca813
  Stored in directory: /Users/egor/Library/Caches/pip/wheels/b9/0d/8a/0729d2e6e3ca2a898ba55201f905da7db3f838a33df5b3fcdd
Successfully built lightfm


Импортируем нужные нам функции из этой библиотеки. На этом этапе сразу же загрузим инструменты оценки модели:



In [84]:
from lightfm import LightFM
from lightfm.cross_validation import random_train_test_split
from lightfm.evaluation import precision_at_k, recall_at_k  

/Users/egor/anaconda3/lib/python3.11/site-packages/lightfm/_lightfm_fast.py:9: UserWarning: LightFM was compiled without OpenMP support. Only a single thread will be used.
  warnings.warn(


Работать мы будем с датасетом goodreads_book.

Goodreads — это сайт, на котором люди могут добавлять книги в каталоги, искать их, изучать аннотации и отзывы. Пользователи также могут создавать сообщества, в которых они рекомендуют друг другу различную литературу, ведут блоги и устраивают обсуждения.
Подгрузим все файлы, относящиеся к этому набору данных:

In [85]:
ratings = pd.read_csv('data/ratings.csv') # Поставленные оценки
books = pd.read_csv('data/books.csv') # Информация о книгах
tags = pd.read_csv('data/tags.csv') # Информация о тегах
book_tags = pd.read_csv('data/book_tags.csv') # Книги с тегами 

Сначала посмотрим на набор данных books: в этих данных есть обычный id книги, а есть id книги в системе Goodreads — этот id отображён в признаке goodreads_book_id. В других данных (book_tags) указан только id книги в системе Goodreads, поэтому нам необходимо добавить туда обычный id.

In [100]:
books

,book_id,goodreads_book_id,best_book_id,work_id,books_count,isbn,isbn13,authors,original_publication_year,original_title,...,ratings_count,work_ratings_count,work_text_reviews_count,ratings_1,ratings_2,ratings_3,ratings_4,ratings_5,image_url,small_image_url
0,1,2767052,2767052,2792775,272,439023483,9.780439e+12,Suzanne Collins,2008.0,The Hunger Games,...,4780653,4942365,155254,66715,127936,560092,1481305,2706317,https://images.gr-assets.com/books/1447303603m...,https://images.gr-assets.com/books/1447303603s...
1,2,3,3,4640799,491,439554934,9.780440e+12,"J.K. Rowling, Mary GrandPré",1997.0,Harry Potter and the Philosopher's Stone,...,4602479,4800065,75867,75504,101676,455024,1156318,3011543,https://images.gr-assets.com/books/1474154022m...,https://images.gr-assets.com/books/1474154022s...
2,3,41865,41865,3212258,226,316015849,9.780316e+12,Stephenie Meyer,2005.0,Twilight,...,3866839,3916824,95009,456191,436802,793319,875073,1355439,https://images.gr-assets.com/books/1361039443m...,https://images.gr-assets.com/books/1361039443s...
3,4,2657,2657,3275794,487,61120081,9.780061e+12,Harper Lee,1960.0,To Kill a Mockingbird,...,3198671,3340896,72586,60427,117415,446835,1001952,1714267,https://images.gr-assets.com/books/1361975680m...,https://images.gr-assets.com/books/1361975680s...
4,5,4671,4671,245494,1356,743273567,9.780743e+12,F. Scott Fitzgerald,1925.0,The Great Gatsby,...,2683664,2773745,51992,86236,197621,606158,936012,947718,https://images.gr-assets.com/books/1490528560m...,https://images.gr-assets.com/books/1490528560s...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9995,9996,7130616,7130616,7392860,19,441019455,9.780441e+12,Ilona Andrews,2010.0,Bayou Moon,...,17204,18856,1180,105,575,3538,7860,6778,https://images.gr-assets.com/books/1307445460m...,https://images.gr-assets.com/books/1307445460s...
9996,9997,208324,208324,1084709,19,067973371X,9.780680e+12,Robert A. Caro,1990.0,Means of Ascent,...,12582,12952,395,303,551,1737,3389,6972,https://s.gr-assets.com/assets/nophoto/book/11...,https://s.gr-assets.com/assets/nophoto/book/50...
9997,9998,77431,77431,2393986,60,039330762X,9.780393e+12,Patrick O'Brian,1977.0,The Mauritius Command,...,9421,10733,374,11,111,1191,4240,5180,https://images.gr-assets.com/books/1455373531m...,https://images.gr-assets.com/books/1455373531s...
9998,9999,8565083,8565083,13433613,7,61711527,9.780062e+12,Peggy Orenstein,2011.0,Cinderella Ate My Daughter: Dispatches from th...,...,11279,11994,1988,275,1002,3765,4577,2375,https://images.gr-assets.com/books/1279214118m...,https://images.gr-assets.com/books/1279214118s...


In [88]:
book_tags

,goodreads_book_id,tag_id,count
0,1,30574,167697
1,1,11305,37174
2,1,11557,34173
3,1,8717,12986
4,1,33114,12716
...,...,...,...
999907,33288638,21303,7
999908,33288638,17271,7
999909,33288638,1126,7
999910,33288638,11478,7


Задание 4.1

Добавьте в набор данных book_tags признак с обычным id книги, используя соответствие обычного id и id в системе Goodreads.

Какой обычный id у книги, которая имеет id 5 в системе Goodreads?



In [90]:
book_tags = book_tags.merge(
    books[['book_id', 'goodreads_book_id']],
    on='goodreads_book_id',
    how='left')
book_tags

,goodreads_book_id,tag_id,count,book_id
0,1,30574,167697,27
1,1,11305,37174,27
2,1,11557,34173,27
3,1,8717,12986,27
4,1,33114,12716,27
...,...,...,...,...
999907,33288638,21303,7,8892
999908,33288638,17271,7,8892
999909,33288638,1126,7,8892
999910,33288638,11478,7,8892


In [91]:
book_tags[book_tags['goodreads_book_id'] == 5]

,goodreads_book_id,tag_id,count,book_id
300,5,11557,40087,18
301,5,11305,39330,18
302,5,8717,17944,18
303,5,33114,12856,18
304,5,30574,11909,18
...,...,...,...,...
395,5,20781,299,18
396,5,32345,298,18
397,5,12600,282,18
398,5,3379,277,18


Задание 4.2

Далее нам необходимо оставить в наборе данных book_tags только те записи, теги для которых есть в данных tags.

Отфильтруйте данные таким образом, чтобы в наборе данных book_tags остались только те строки, в которых находятся теги, информация о которых есть в наборе данных tags.

Сколько объектов осталось?



In [93]:
book_tags = book_tags.merge(tags, on='tag_id', how='right')
book_tags

,goodreads_book_id,tag_id,count,book_id,tag_name
0,67,509,17,3504,19th-century
1,93,509,72,639,19th-century
2,264,509,175,1783,19th-century
3,295,509,241,293,19th-century
4,304,509,112,5915,19th-century
...,...,...,...,...,...
300733,17235026,33268,1132,886,zombies
300734,17333174,33268,10,9712,zombies
300735,18007535,33268,23,3429,zombies
300736,18667307,33268,9,5074,zombies


Отлично, мы подготовили информацию о тегах книг — это будет метаинформацией для построения рекомендательной системы. Теперь нам необходимо подготовить данные о взаимодействии пользователей и книг. Для этого нам понадобится файл ratings.

Оба набора данных (и про взаимодействия, и про метаинформацию) необходимо преобразовать в разрежённые матрицы. Это можно сделать с помощью специальной функции из модуля scipy:

In [94]:
from scipy.sparse import csr_matrix

Нам важно преобразовать данные в специальный формат, в котором хранятся разрежённые матрицы — будем использовать формат Compressed Sparse Row (CSR), подразумевающий подсчёт кумулятивной суммы количества элементов в строке вместо индексов строк.

Здесь хранится информация о том, сколько суммарно ненулевых элементов в данной строке и выше, индексы столбцов с ненулевыми значениями, сами значения и размерность матрицы.

В первой строке обозначено, сколько накоплено ненулевых значений (в первой строке — суммарно 1, после второй строки — суммарно 4, после третьей строки — суммарно 4, после четвёртой строки — суммарно 6).
Во второй строке показано, в каком столбце находится ненулевое значение.
В третьей строке указаны сами значения.
Осуществляем преобразование следующим образом:

In [105]:
ratings_matrix = csr_matrix((ratings.rating,(ratings.user_id, ratings.book_id))) 
# Передаём в качестве аргументов в функцию выставленный рейтинг (это будут значения матрицы), 
# а также id пользователя и id книги (это будут индексы для строк и столбцов матрицы)

Теперь нам необходимо составить матрицу с метаданными. В качестве индексов будут выступать id книги и id тега, и если у этой книги есть рассматриваемый тег, то на пересечении соответствующих строки и столбца будет выставлена единица.



In [108]:
meta_matrix  = csr_matrix(([1]*len(book_tags),(book_tags.book_id, book_tags.tag_id))) 

Задание 4.4

Давайте проверим, что всё получилось правильно.

Каково среднее арифметическое значений разрежённой матрицы с рейтингами? Ответ округлите до трёх знаков после точки-разделителя.

In [111]:
ratings_matrix.mean()

0.007086188900997592

Отлично, данные подготовлены — теперь настало время определить модель, которую мы будем использовать. Сделаем это следующим образом:



In [112]:
model = LightFM(
    loss='warp-kos', # Определяем функцию потерь
    random_state=42, # Фиксируем случайное разбиение
    learning_rate=0.05, # Темп обучения
    no_components=100 # Размерность вектора для представления данных в модели
)

В качестве функции потерь мы выбрали значение 'warp', хотя, разумеется, это не единственный вариант. В модуле LightFM представлены следующие функции потерь:

'logistic' — логистическая функция. Полезна в случаях, когда есть как положительные, так и отрицательные взаимодействия, например 1 и -1.
'bpr' — байесовский персонализированный рейтинг. Можно применять, когда присутствуют только положительные взаимодействия.
'warp' — парный взвешенный приблизительный ранг. Используется, если необходимо повысить качество именно в верхней части списка рекомендаций.
'warp-kos' — модификация warp.

Разобьём данные на обучающую и тестовую выборки:



In [113]:
train, test = random_train_test_split(
    ratings_matrix, # Общая выборка
    test_percentage=0.2, # Размер тестовой выборки
    random_state=42 # Генератор случайных чисел
)

Теперь обучим модель на наших данных о взаимодействии, также используя метаданные о книгах. Для этого воспользуемся методом fit(). В этот метод передадим обучающую выборку, признаки товаров — item_features, количество эпох обучения (сколько раз мы будем показывать модели исходный датасет, чтобы она лучше выучила данные) — epochs, а также параметр verbose для отслеживания процесса обучения:

In [114]:
model = model.fit(
    train, # Обучающая выборка
    item_features=meta_matrix, # Признаки товаров
    epochs=10, # Количество эпох
    verbose=True # Отображение обучения
)

Epoch: 100%|██████████| 10/10 [01:40<00:00, 10.01s/it]


Задание 4.5

Оцените качество полученной модели с помощью функции precision_at_k, передав в неё три аргумента: модель, тестовые данные и обозначение метаданных (item_features = meta_matrix).

Примечание. Процесс расчёта метрик рекомендательной системы также является довольно затратным по времени. Для ускорения этого процесса вы можете передать параметр num_threads, чтобы указать количество потоков процессора, используемых для вычислений.

In [116]:
qlty = precision_at_k(model, test, item_features=meta_matrix)
qlty.mean()

0.023578687

В рекомендательных системах метрики интерпретируются иначе, чем в задачах классификации. Показатели точности РС считаются хорошими, если они находятся в районе 0.1-0.3.

У нас получился не слишком высокий, но довольно неплохой результат. Чтобы его улучшить, можно попробовать следующее:

Поработать над предобработкой данных, добавив в них дополнительную информацию о товарах. Также можно попробовать воспользоваться иным способом создания разреженной матрицы, например, форматом coo_matrix() или csc_matrix(), которые также входят в библиотеку scipy. Подробнее почитать о них вы можете здесь.
Поиграться с параметрами модели LightFM — поуправлять темпом обучения (learning_rate), размерностью вектора для представления (no_components), количеством эпох обучения (epochs) и функцией потерь (loss).
Примечание. Для предсказания рейтинга нового пользователя можно воспользоваться методом predict():

In [118]:
scores = model.predict(<индекс интересующего пользователя>, np.arange(n_items), user_features=new_user_feature)

NameError: name 'n_items' is not defined

## Современные методы: глубокое обучение

Глубокое обучение (Deep Learning, DL) — это современное и эффективное решение для многих задач машинного обучения, таких как компьютерное зрение или обработка естественного языка. Deep Learning во многих случаях превосходит классические методы, которые мы рассматривали ранее. Поэтому в последнее время глубокое обучение всё чаще применяется и в рекомендательных системах. Многие крупные компании, такие как AirBnB, Google, Home Depot, LinkedIn и Pinterest, используют рекомендательные системы, построенные именно на основе глубокого обучения.

Преимущества использования нейронных сетей:

Как правило, DL-модели дают более высокое качество. Стандартные ML-модели проигрывают глубокому обучению, особенно в ситуациях с большим объёмом данных.
DL-модели обладают большей гибкостью. В рамках одной модели вы можете получить ответы на такие вопросы, как «Добавит ли пользователь товар в корзину?», «Начнёт ли он оформление заказа с этим товаром?» или «Купит ли он этот товар?».
Можно включать в модель данные совершенно разных типов, в т. ч. текстовые данные (используя на них все инструменты NLP) или изображения (используя свёрточные нейронные сети).
Существует множество архитектур нейронных сетей, которые можно использовать для разработки рекомендательных систем. Сейчас мы рассмотрим простейшую архитектуру, чтобы в целом разобраться с принципом создания РС с использованием глубокого обучения.

Прежде чем погрузиться в архитектуру модели, давайте познакомимся с концепцией эмбеддингов.

Эмбеддинг — это пространство низкой размерности, которое отражает взаимосвязь векторов из пространства более высокой размерности.

Чтобы лучше понять эту концепцию, давайте рассмотрим пример создания эмбеддингов пользователей.

Представим, что у нас есть различные данные о пользователях, которые мы хотим преобразовать в векторы с двумя координатами, отражающие всего две характеристики:

степень симпатии к триллерам;
степень симпатии к мелодрамам.

Теперь давайте представим некоторого пользователя Алису. Алиса предпочитает смотреть триллеры и не очень любит мелодрамы. Если в зависимости от её действий и оценок фильмов мы сможем оценить её любовь к триллерам и нелюбовь к мелодрамам в численном эквиваленте, то мы получим вектор, характеризующий предпочтения Алисы в плане жанров:

Рассмотрим ещё одного пользователя — Машу. Маша в целом является киноманом, так что примерно одинаково любит и триллеры, и мелодрамы. По аналогии с Алисой попробуем создать вектор, который характеризует предпочтения Маши.

Создав такие векторы-представления для всех пользователей, мы получим полный набор эмбеддингов.

По сути, эмбеддинги уменьшают размерность данных, оставляя осмысленность в их отображении. При таком преобразовании пользователи, предпочтения которых похожи, находятся рядом в плоскости (или пространстве), а пользователи, предпочтения которых отличаются, находятся далеко друг от друга.

Конечно, необязательно использовать для формирования эмбеддинга только два числа — для вектора можно взять любое количество компонент. Причём чем больше будет измерений, тем выше будет точность представления всех особенностей пользователя и, как следствие, выше будет точность модели. Однако следует понимать, что такое увеличение точности происходит за счёт роста сложности модели и увеличения количества времени, необходимого для её обучения.
Ровно таким же образом мы можем создать эмбеддинги и для фильмов, отобразив информацию о них в пространство меньшей размерности.

Теперь, когда мы разобрались с эмбеддингами, можно переходить к самой архитектуре нейронной сети. Рассмотрим следующий пример архитектуры:

В качестве входных данных для нейронной сети мы передаём вектор характеристик пользователя и вектор для фильма (Inputs на схеме выше).

Из обоих этих векторов получаются эмбеддинги, о которых мы поговорили ранее (обозначены как Embedded User Vector и Embedded Item Vector). Далее эти эмбеддинги проходят через несколько полносвязных слоёв, на выходе из которых они преобразуются в вектор-предсказание. Например, на схеме выше видно, что вероятность первого класса (показывает, что элемент нерелевантен) — 0.2, а второго (показывает, что элемент релевантен) — 0.8. Следовательно, мы делаем выбор в пользу второго и рекомендуем этот продукт пользователю. Собственно, по такому алгоритму и обучается эта нейронная сеть.

Конечно, наше знакомство с глубоким обучением получилось очень поверхностным, но сейчас для вас важно просто понимать его принцип.

Давайте разберём несложную задачу, при решении которой мы обучим настоящую нейронную сеть и используем её для создания рекомендаций.
Мы будем использовать модуль tensorflow, в котором реализовано много полезных методов для имплементации (внедрения) нейронных сетей. Установим его:


In [119]:
!pip install tensorflow


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 189.3/189.3 MB 14.1 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 126.5/126.5 kB 12.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.5/57.5 kB 6.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.6/2.6 MB 14.2 MB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 20.6/20.6 MB 14.6 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 11.4 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.5/65.5 kB 8.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 400.3/400.3 kB 13.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.9/8.9 MB 10.5 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.6/5.6 MB 21.0 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 440.8/440.8 kB 15.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 1

Для начала импортируем из него функции, которые понадобятся нам для решения задачи:

In [120]:
from tensorflow.keras.layers import Input, Embedding, Flatten, Dot, Dense, Concatenate
from tensorflow.keras.models import Model

Мы будем использовать данные из предыдущего юнита, но лишь те, которые содержат информацию об оценках, выставленных книгам пользователями. Загрузим данные:

In [121]:
df = pd.read_csv('data/ratings.csv')

Задание 5.1

Разбейте данные на обучающую и тестовую выборки в отношении 4:1. В качестве значения параметра random_state возьмите число 42.

Сколько объектов теперь находится в обучающей выборке?

In [125]:
from sklearn.model_selection import train_test_split

train_df, test_df = train_test_split(df, test_size=0.2, random_state=42)
train_df.shape[0]

785404

Задание 5.2

Запишите количество уникальных книг в переменную n_books.

Сколько в наборе данных уникальных книг?

In [128]:
n_books = df.book_id.nunique()
n_books

10000

Задание 5.3

Запишите количество уникальных пользователей в переменную n_users.

Сколько в наборе данных уникальных пользователей?

In [131]:
n_users = df.user_id.nunique()
n_users

53424

В первую очередь нам необходимо создать эмбеддинги для книг и пользователей. Создаём эмбеддинги для книг:

In [132]:
book_input = Input(shape=[1], name="Book-Input")
book_embedding = Embedding(n_books+1, 5, name="Book-Embedding")(book_input)
book_vec = Flatten(name="Flatten-Books")(book_embedding)

Сначала мы задаём размерность входного слоя. После этого определяем размер эмбеддинга — в данном случае снижаем размерность до 5. Далее мы разворачиваем результат в массив с одним измерением с помощью слоя Flatten().

Делаем то же самое для пользователей:

In [133]:
user_input = Input(shape=[1], name="User-Input")
user_embedding = Embedding(n_users+1, 5, name="User-Embedding")(user_input)
user_vec = Flatten(name="Flatten-Users")(user_embedding)

Теперь, когда мы создали представления как для книг, так и для пользователей, нам необходимо соединить их:



In [134]:
conc = Concatenate()([book_vec, user_vec])


Далее начинаем «собирать» нашу нейронную сеть из слоёв. Dense обозначает полносвязный слой. Также мы обозначаем для него количество нейронов и данные, которые идут на вход.

In [135]:
fc1 = Dense(128, activation='relu')(conc)
fc2 = Dense(32, activation='relu')(fc1)
out = Dense(1)(fc2)

Собираем модель — передаём входные данные для книг и пользователей, а также архитектуру нейронной сети:



In [136]:
model2 = Model([user_input, book_input], out)



Также нам необходимо задать алгоритм оптимизации и метрику, которую мы будем оптимизировать. В данном случае будем использовать метод adam и хорошо известную вам среднеквадратичную ошибку:

In [137]:
model2.compile(optimizer = 'adam',loss =  'mean_squared_error')

Теперь будем обучать нашу модель:

In [139]:
history = model2.fit([train_df.user_id, train_df.book_id], train_df.rating, epochs=5, verbose=1)

Epoch 1/5
24544/24544 [==============================] - 40s 2ms/step - loss: 0.7984
Epoch 2/5
24544/24544 [==============================] - 40s 2ms/step - loss: 0.6870
Epoch 3/5
24544/24544 [==============================] - 40s 2ms/step - loss: 0.6551
Epoch 4/5
24544/24544 [==============================] - 40s 2ms/step - loss: 0.6296
Epoch 5/5
24544/24544 [==============================] - 40s 2ms/step - loss: 0.6072


В параметр эпох передаём значение 5: у нас будет реализовано пять эпох — пять обучений нейронной сети. На каждой из эпох обновляются веса для минимизации ошибки.

Теперь можно оценить качество:

In [140]:
model2.evaluate([test_df.user_id, test_df.book_id], test_df.rating)


6136/6136 [==============================] - 2s 344us/step - loss: 0.7106


0.7105711102485657

Примечание. К сожалению, результаты этого алгоритма нельзя зафиксировать стандартным ramdom_state, к которому мы привыкли: применяемые методы не используют такой параметр. Поэтому мы опустим здесь сравнение результатов, однако посмотрим, как можно настроить нейронную сеть.

Обычно для улучшения качества модели каким-то образом модифицируют нейронную сеть: дополняют её, увеличивают время обучения. Добавим ещё один полносвязный слой с восемью нейронами после полносвязного слоя с 32 нейронами. Обучим нейронную сеть, реализовав десять эпох:

In [142]:
fc1 = Dense(128, activation='relu')(conc)
fc2 = Dense(32, activation='relu')(fc1)
fc3 = Dense(8, activation='relu')(fc2)
out = Dense(1)(fc3)

model2 = Model([user_input, book_input], out)
model2.compile('adam', 'mean_squared_error')
result = model2.fit([train_df.user_id, train_df.book_id], train_df.rating, epochs=10, verbose=1)
model2.evaluate([test_df.user_id, test_df.book_id], test_df.rating)

Epoch 1/10
24544/24544 [==============================] - 38s 2ms/step - loss: 0.6446
Epoch 2/10
24544/24544 [==============================] - 38s 2ms/step - loss: 0.5744
Epoch 3/10
24544/24544 [==============================] - 38s 2ms/step - loss: 0.5550
Epoch 4/10
24544/24544 [==============================] - 37s 2ms/step - loss: 0.5376
Epoch 5/10
24544/24544 [==============================] - 37s 2ms/step - loss: 0.5232
Epoch 6/10
24544/24544 [==============================] - 37s 2ms/step - loss: 0.5113
Epoch 7/10
24544/24544 [==============================] - 38s 2ms/step - loss: 0.5014
Epoch 8/10
24544/24544 [==============================] - 38s 2ms/step - loss: 0.4928
Epoch 9/10
24544/24544 [==============================] - 39s 2ms/step - loss: 0.4856
Epoch 10/10
6136/6136 [==============================] - 2s 303us/step - loss: 0.7818


0.7818419933319092

Качество получившейся модели не будет выше качества предыдущей, так как усложнение сети или увеличение количества эпох не всегда даёт высокое качество. Здесь главное, что вы научились корректировать архитектуру нейронной сети.

